In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

In [2]:
movie_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/movie.csv')
rating_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/rating.csv')
link_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/link.csv')
tag_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/tag.csv')
genome_tags_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/genome_tags.csv')
genome_scores_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/genome_scores.csv')

In [3]:
#this is not the same combined small as collaborative
combined_small = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/tags_mvoies_combined_small.csv')

In [4]:
#take a copy of the data frame
df_tag = genome_tags_df.copy(deep = True)
df_score = genome_scores_df.copy(deep = True)
combined_copy_tag = combined_small.copy(deep = True)

In [5]:
#count how many times each movie tagged with different tag
tags_df_counted = pd.DataFrame((tag_df.groupby(['movieId','tag'])['userId'].count())).reset_index()
tags_df_counted.rename(columns={'userId':'count'},inplace=True)

In [6]:
#fucntion creates a data frame with movie id and taglist as columns
def creat_tag_list(tags_df_counted):
  df = pd.DataFrame([[0,0]],columns= ['movieId','tag_list'])
  filtered_unique_id = tags_df_counted['movieId'].unique().tolist()
  for id in filtered_unique_id:
    tags_list = tags_df_counted.loc[tags_df_counted['movieId'] == id]['tag'].values.tolist()
    df_dummy = pd.DataFrame([[id,tags_list]],columns=['movieId','tag_list'])
    df = df.append(df_dummy)

  #reset the index and drop nans  
  tag_list_combined = df.reset_index().drop(columns=['index'])
  tag_list_combined.drop(tag_list_combined.index[0],inplace=True) 

  return tag_list_combined

In [7]:
combined_copy_tag.drop(columns=['Unnamed: 0'],inplace=True)
tag_list_combined = creat_tag_list(tags_df_counted)
tag_list_combined.head(5)

,movieId,tag_list
1,1,"[2009 reissue in Stereoscopic 3-D, 3D, 55 movi..."
2,2,"[CLV, Children, Chris Van Allsburg, Dynamic CG..."
3,3,"[CLV, Funniest Movies, Howard Deutch, Jack Lem..."
4,4,"[CLV, characters, chick flick, revenge]"
5,5,"[CLV, Comedy, Diane Keaton, Fantasy, Steve Mar..."


In [8]:
#how many time a tag was used
combined_grouped = pd.DataFrame((combined_copy_tag.groupby(['movieId','genres'])\
                                 ['userId'].count())).reset_index()
combined_grouped.rename(columns={'userId':'count'},inplace=True)
combined_grouped.head(5)

,movieId,genres,count
0,1,Adventure|Animation|Children|Comedy|Fantasy,49695
1,2,Adventure|Children|Fantasy,22243
2,3,Comedy|Romance,12735
3,4,Comedy|Drama|Romance,2756
4,5,Comedy,12161


In [9]:
#movie tag list and genres are combined together
merged = pd.concat([tag_list_combined,combined_grouped.drop(columns=['movieId'])],axis=1)
merged.dropna(inplace=True)
merged.head(5)

,movieId,tag_list,genres,count
1,1.0,"[2009 reissue in Stereoscopic 3-D, 3D, 55 movi...",Adventure|Children|Fantasy,22243.0
2,2.0,"[CLV, Children, Chris Van Allsburg, Dynamic CG...",Comedy|Romance,12735.0
3,3.0,"[CLV, Funniest Movies, Howard Deutch, Jack Lem...",Comedy|Drama|Romance,2756.0
4,4.0,"[CLV, characters, chick flick, revenge]",Comedy,12161.0
5,5.0,"[CLV, Comedy, Diane Keaton, Fantasy, Steve Mar...",Action|Crime|Thriller,23899.0


In [10]:
#changing the types from float to int and list ot string
merged['movieId'] = merged['movieId'].astype(int)
merged['userId'] = merged['count'].astype(int)
merged['tag_list'] = merged['tag_list'].astype(str)
merged['genres'] = merged['genres'].astype(str)

#creating a column which is genres and tag_list combined
merged['Bag_of_words'] = merged['tag_list'] + merged['genres']
merged.head(5)

,movieId,tag_list,genres,count,userId,Bag_of_words
1,1,"['2009 reissue in Stereoscopic 3-D', '3D', '55...",Adventure|Children|Fantasy,22243.0,22243,"['2009 reissue in Stereoscopic 3-D', '3D', '55..."
2,2,"['CLV', 'Children', 'Chris Van Allsburg', 'Dyn...",Comedy|Romance,12735.0,12735,"['CLV', 'Children', 'Chris Van Allsburg', 'Dyn..."
3,3,"['CLV', 'Funniest Movies', 'Howard Deutch', 'J...",Comedy|Drama|Romance,2756.0,2756,"['CLV', 'Funniest Movies', 'Howard Deutch', 'J..."
4,4,"['CLV', 'characters', 'chick flick', 'revenge']",Comedy,12161.0,12161,"['CLV', 'characters', 'chick flick', 'revenge'..."
5,5,"['CLV', 'Comedy', 'Diane Keaton', 'Fantasy', '...",Action|Crime|Thriller,23899.0,23899,"['CLV', 'Comedy', 'Diane Keaton', 'Fantasy', '..."


In [11]:
merged['Bag_of_words'].values[0]

'[\'2009 reissue in Stereoscopic 3-D\', \'3D\', \'55 movies every kid should see--Entertainment Weekly\', \'Animation\', \'BD-Video\', \'Best of Rotten Tomatoes: All Time\', \'Buzz Lightyear\', \'CG animation\', \'CGI\', \'CLV\', \'Cartoon\', \'DARING RESCUES\', \'DVD-Video\', \'Disney\', \'Disney animated feature\', \'Engaging\', \'HEROIC MISSION\', \'John Lasseter\', \'National Film Registry\', \'Pixar\', \'Pixar animation\', \'TOYS COME TO LIFE\', \'Tim Allen\', \'Tom Hanks\', "Tumey\'s To See Again", "Tumey\'s VHS", \'TÃ©a Leoni does not star in this movie\', \'UNLIKELY FRIENDSHIPS\', \'USA\', \'Want\', \'Watched\', \'Woody\', \'action figure\', \'action figures\', \'adventure\', \'almost favorite\', \'animated\', \'animation\', \'avi\', \'bright\', \'buddy movie\', \'buy\', \'cgi\', \'children\', \'classic\', \'clever\', \'comedy\', \'computer animation\', \'cute\', \'disney\', \'dolls\', "erlend\'s DVDs", \'family\', \'family film\', \'fanciful\', \'fantasy\', \'first cgi film\',

In [12]:
#removing non alphanumeric and double spaces
#changing from upper to lower
def text_cleaning(row):
  alpha_numeric = re.sub("\W"," ",row)
  double_space = re.sub(' +',' ',alpha_numeric)
  remove_space = double_space.strip()
  lower = remove_space.lower()
  return lower

merged['Bag_of_words'] = merged['Bag_of_words'].apply(text_cleaning)
merged.drop(columns=['genres','count','tag_list'],inplace=True)

In [13]:
merged.tail(5)

,movieId,userId,Bag_of_words
499,565,6583,claudio brook criterion frederico luppi guille...
500,566,10219,writers comedy musical romance
501,567,3033,almodovar clv pedro almodovar pedro almodã³var...
502,568,1546,directorial debut comedy musical
503,569,7983,andrew scheinman baseball drama romance


In [14]:
movie_names = combined_small[['movieId','title']].drop_duplicates()
#mergin movie names and bag_of_word we created
final = pd.merge(left=merged,right=movie_names,on='movieId',how='right')
final.dropna(inplace=True)
final.drop(columns=['movieId','userId'],inplace=True)
final['title'] = final['title'].apply(lambda x: x.lower())

In [15]:
final.head(5)

,Bag_of_words,title
0,2009 reissue in stereoscopic 3 d 3d 55 movies ...,toy story (1995)
1,clv children chris van allsburg dynamic cgi ac...,jumanji (1995)
2,clv funniest movies howard deutch jack lemmon ...,grumpier old men (1995)
3,clv characters chick flick revenge comedy,waiting to exhale (1995)
4,clv comedy diane keaton fantasy steve martin t...,father of the bride part ii (1995)


In [16]:
#calculating cosime similarity
count = CountVectorizer()
count_matrix = count.fit_transform(final['Bag_of_words'])
cosine_sim = cosine_similarity(count_matrix,count_matrix)

In [17]:
#function to recommend movies
indices = pd.Series(final['title'])
def recommend(title, cosine_sim = cosine_sim):
    recommended_movies = []
    idx = indices[indices.str.contains(title)].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_movies.append(list(final['title'])[i])
        
    return recommended_movies

In [18]:
recommend('toy story')

['lion king, the (1994)',
 'nightmare before christmas, the (1993)',
 'jurassic park (1993)',
 'pulp fiction (1994)',
 'jumanji (1995)',
 'star wars: episode iv - a new hope (1977)',
 'apollo 13 (1995)',
 'swan princess, the (1994)',
 'forrest gump (1994)',
 'shawshank redemption, the (1994)']